# 設定

In [15]:
import time
import hmac
import hashlib
import base64
import json
import logging
import requests


class CoincatchClient:
    def __init__(self, api_key: str, secret_key: str, passphrase: str, memo: str = None):
        """
        Coincatch API 客戶端
        :param api_key: API Key
        :param secret_key: Secret Key
        :param passphrase: API Passphrase
        :param memo: 備註（可選）
        """
        self.api_key = api_key
        self.secret_key = secret_key
        self.passphrase = passphrase
        self.memo = memo
        self.base_url = "https://api.coincatch.com"
        self.logger = logging.getLogger(__name__)

    # === 通用簽名 ===
    def _get_signed_headers(self, method: str, path: str, query: str = "", body: dict = None):
        """
        建立 Coincatch API 所需 Header
        簽名規則：sign = base64(HMAC_SHA256(secret, timestamp + method + requestPath + body))
        """
        timestamp = str(int(time.time() * 1000))
        body_str = json.dumps(body, separators=(',', ':')) if body else ""
        message = timestamp + method.upper() + path + query + body_str

        signature = base64.b64encode(
            hmac.new(
                self.secret_key.encode("utf-8"),
                message.encode("utf-8"),
                hashlib.sha256
            ).digest()
        ).decode()

        headers = {
            "ACCESS-KEY": self.api_key,
            "ACCESS-SIGN": signature,
            "ACCESS-TIMESTAMP": timestamp,
            "ACCESS-PASSPHRASE": self.passphrase,
            "locale": "en-US",
            "Content-Type": "application/json"
        }
        return headers

    # === 查詢合約帳戶餘額 ===
    def get_balance(self, product_type: str = "umcbl"):
        """
        查詢合約帳戶餘額 (USDT-M)
        :param product_type: 產品類型, 預設 'umcbl' (USDT 永續)
        :return: USDT 可用餘額 (float)
        """
        path = "/api/mix/v1/account/accounts"
        query = f"?productType={product_type}"
        url = self.base_url + path + query
        headers = self._get_signed_headers("GET", path, query)

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            print(data)

            if data.get("code") == "00000":
                account = data["data"][0]
                available = float(account["available"])
                self.logger.info(f"USDT balance: {available}")
                return available
            else:
                self.logger.error(f"API error: {data}")
                return None
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Request failed: {e}")
            return None
        except ValueError:
            self.logger.error("Failed to decode JSON response.")
            return None

    # === 查詢持倉 ===
    def get_open_positions(self, symbol: str = None, product_type: str = "umcbl"):
        """
        查詢當前持倉
        """
        path = "/api/mix/v1/position/allPosition"
        query = f"?productType={product_type}"
        if symbol:
            query += f"&symbol={symbol}"
        url = self.base_url + path + query

        headers = self._get_signed_headers("GET", path, query)

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            data = response.json()
            print(data)

            if data.get("code") == "00000":
                return data.get("data", [])
            else:
                self.logger.error(f"API error: {data}")
                return None
        except Exception as e:
            self.logger.error(f"Failed to get open positions: {e}")
            return None

    # === 開倉 ===
    def place_order(self, symbol: str, side: str, margin: float, leverage: int,
                    tp_price: float = None, sl_price: float = None, product_type: str = "umcbl"):
        """
        下單（開倉）
        """
        path = "/api/mix/v1/order/placeOrder"
        url = self.base_url + path
        method = "POST"

        side_map = {"long": "open_long", "short": "open_short"}
        if side.lower() not in side_map:
            self.logger.error("Invalid side. Must be 'long' or 'short'.")
            return None

        payload = {
            "symbol": symbol,
            "marginCoin": "USDT",
            "size": str(margin),
            "side": side_map[side.lower()],
            "orderType": "market",
            "leverage": str(leverage),
            "productType": product_type
        }

        if tp_price:
            payload["presetTakeProfitPrice"] = str(tp_price)
        if sl_price:
            payload["presetStopLossPrice"] = str(sl_price)

        headers = self._get_signed_headers(method, path, "", payload)

        try:
            response = requests.post(url, headers=headers, data=json.dumps(payload))
            response.raise_for_status()
            data = response.json()
            if data.get("code") == "00000":
                self.logger.info(f"Order placed successfully: {data}")
                return data
            else:
                self.logger.error(f"API error: {data}")
                return None
        except Exception as e:
            self.logger.error(f"Failed to place order: {e}")
            return None

    # === 平倉 ===
    def close_position(self, symbol: str, side: str, product_type: str = "umcbl"):
        """
        平倉
        """
        path = "/api/mix/v1/order/placeOrder"
        url = self.base_url + path
        method = "POST"

        side_map = {"long": "close_long", "short": "close_short"}
        if side.lower() not in side_map:
            self.logger.error("Invalid side. Must be 'long' or 'short'.")
            return None

        payload = {
            "symbol": symbol,
            "marginCoin": "USDT",
            "side": side_map[side.lower()],
            "orderType": "market",
            "productType": product_type
        }

        headers = self._get_signed_headers(method, path, "", payload)

        try:
            response = requests.post(url, headers=headers, data=json.dumps(payload))
            response.raise_for_status()
            data = response.json()
            if data.get("code") == "00000":
                self.logger.info(f"Position closed successfully: {data}")
                return data
            else:
                self.logger.error(f"API error: {data}")
                return None
        except Exception as e:
            self.logger.error(f"Failed to close position: {e}")
            return None


In [16]:
import os
from dotenv import load_dotenv
load_dotenv()

client = CoincatchClient(
    api_key=os.getenv("COINCATCH_API_KEY"),
    secret_key=os.getenv("COINCATCH_SECRET_KEY"),
    passphrase=os.getenv("COINCATCH_API_PASSPHRASE")
)

balance = client.get_balance()
print("USDT 餘額:", balance)

positions = client.get_open_positions()
print("持倉:", positions)

order = client.place_order('ETHUSDT','long',10,10)
print('下單:',order)

close = client.close_position('ETHUSDT','long')
print('關倉:',close)


{'code': '00000', 'msg': 'success', 'requestTime': 1762832249843, 'data': [{'marginCoin': 'USDT', 'locked': '0', 'available': '0', 'crossMaxAvailable': '0', 'fixedMaxAvailable': '0', 'maxTransferOut': '0', 'equity': '0', 'usdtEquity': '0', 'btcEquity': '0', 'crossRiskRate': '0', 'unrealizedPL': '0', 'bonus': '0', 'crossedUnrealizedPL': None, 'isolatedUnrealizedPL': None}]}
USDT 餘額: 0.0
{'code': '00000', 'msg': 'success', 'requestTime': 1762832249954, 'data': []}
持倉: []


Failed to place order: 400 Client Error: Bad Request for url: https://api.coincatch.com/api/mix/v1/order/placeOrder
Failed to close position: 400 Client Error: Bad Request for url: https://api.coincatch.com/api/mix/v1/order/placeOrder


下單: None
關倉: None
